In [ ]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.utils import np_utils
from keras import optimizers

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import random

In [ ]:
train = pd.read_csv('/Users/xiaoyudai/Documents/Paper3/Data/Brain_ES_chr18/crf/train_mlp', 
                    sep='\t', header=None).values
test = pd.read_csv('/Users/xiaoyudai/Documents/Paper3/Data/Brain_ES_chr18/crf/test_mlp', 
                    sep='\t', header=None).values

In [13]:
train.shape

(64716, 34)

In [ ]:
test.shape

In [ ]:
train[1,:]

In [ ]:
def normaling_feature(x_train, x_test):
    dt = np.concatenate((x_train, x_test), axis=0)
    scaler = MinMaxScaler(feature_range=(0,1))
    scaler = scaler.fit(dt)
    scaled_dt = scaler.transform(dt)
    return scaled_dt[:x_train.shape[0], :], scaled_dt[x_train.shape[0]:, :]

In [9]:
x_train = train[:,0:-1].astype(np.float)
y_train = train[:,-1].astype(np.float)

x_test = test[:,0:-1].astype(np.float)
y_test = test[:,-1].astype(np.float)

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [ ]:
scaled_x_train, scaled_x_test = normaling_feature(x_train, x_test)

In [ ]:
idx1 = np.where(y_train[:,1] == 1)[0]
idx0 = np.where(y_train[:,1] == 0)[0]
print(idx0.shape, idx1.shape)

x_train_upsample = scaled_x_train
y_train_upsample = y_train
y_train_fdr_upsample = y_train_fdr

for i in range(10):
    x_train_upsample = np.concatenate((x_train_upsample, scaled_x_train[idx1,:]), axis=0)
    y_train_upsample = np.concatenate((y_train_upsample, y_train[idx1,:]), axis=0)
    y_train_fdr_upsample = np.concatenate((y_train_fdr_upsample, y_train_fdr[idx1]), axis=0)

In [ ]:
y_train_upsample[:,1].mean()

In [ ]:
idx1 = np.where(y_train[:,1] == 1)[0]
idx0 = np.where(y_train[:,1] == 0)[0]
idx0_sample = random.sample(idx0,idx1.shape[0])
idx_sample = np.concatenate((idx0_sample,idx1))
scaled_x_train_sample, y_train_sample = scaled_x_train[idx_sample,], y_train[idx_sample,]

In [21]:
model = Sequential()
model.add(Dense(1024, input_shape=(33,), activation='relu'))
# model.add((Dropout(0.2)))
model.add(Dense(1024, activation='relu'))
# model.add((Dropout(0.2)))
model.add(Dense(512, activation='relu'))
# model.add((Dropout(0.2)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
# model.add(Dense(1, activation='linear'))
model.add(Dense(2, activation='softmax'))

In [33]:
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['binary_accuracy', 'recall'])
# model.compile(optimizer=adam, loss='mean_squared_error')

In [ ]:
model.summary()

In [35]:
model.fit(x_train, y_train, nb_epoch=5, batch_size = 4096)

Epoch 1/5
64716/64716 [==============================] - 16s - loss: 5.2520e-04 - binary_accuracy: 0.9998 - recall: 0.9998    
Epoch 2/5
64716/64716 [==============================] - 16s - loss: 6.8913e-04 - binary_accuracy: 0.9998 - recall: 0.9998    
Epoch 3/5
64716/64716 [==============================] - 16s - loss: 6.1414e-04 - binary_accuracy: 0.9998 - recall: 0.9998    
Epoch 4/5
64716/64716 [==============================] - 17s - loss: 4.3974e-04 - binary_accuracy: 0.9998 - recall: 0.9998    
Epoch 5/5
64716/64716 [==============================] - 17s - loss: 6.1035e-04 - binary_accuracy: 0.9998 - recall: 0.9998    


In [2]:
model.save_weights('/Users/xiaoyudai/Documents/Paper3/Data/code/dnn/model')
# model.load_weights('/Users/xiaoyudai/Documents/Paper3/Data/code/dnn/model')

NameError: name 'model' is not defined

In [ ]:
model.evaluate(scaled_x_test, y_test)

In [37]:
pred_prob = model.predict(x_test)

In [38]:
# pred_prob = model.predict(scaled_x_test)
pred = pred_prob[:,1] > 0.5
truth = y_test[:,1]
confusion_matrix(pred, truth)

array([[290338,  14356],
       [  8252,  10637]])

In [40]:
np.savetxt('/Users/xiaoyudai/Documents/Paper3/Data/Brain_ES_chr18/crf/result_mlp', pred_prob)

In [26]:
pred_prob

array([[  9.99999881e-01,   1.49358954e-07],
       [  9.99999881e-01,   1.49273518e-07],
       [  0.00000000e+00,   1.00000000e+00],
       ..., 
       [  1.00000000e+00,   1.96619787e-30],
       [  1.00000000e+00,   2.55404852e-29],
       [  1.00000000e+00,   2.61501504e-29]], dtype=float32)